<img src="images/simple_splitter.png">

In [88]:
import pandas as pd
import numpy as np
from datetime import date
import json

In [89]:
def read_discount_features_csv(csv):
    discounted = pd.read_csv(csv, na_values=["?", " ", "NA", "na"])
    #for some days there are 2 rows for each medicine in each city so keeping 
    discounted = discounted.sort_values("discounted", ascending = False) 
    discounted = discounted.drop_duplicates(subset = ["date", "city", "medicine"], keep='first')
    discounted = discounted.sort_index()
    discounted["date"] = pd.to_datetime(discounted["date"])
    return discounted

In [90]:
def read_train_csv(csv, city_id):
    train_df = pd.read_csv(csv, na_values=["?", " ", "NA", "na"])
    train = train_df[train_df["city"] == int(city_id)].copy()
    train = train.sort_values("sales", ascending = False) 
    train = train.drop_duplicates(subset = ["year", "month", "day", "city", "medicine"], keep='first')
    train = train.sort_index()
    return train

In [91]:
def read_city_dict(jsonfile):
    with open(jsonfile) as f: 
        cities = json.load(f)
    return {int(key):value for key, value in cities.items()}

In [92]:
def read_test_csv(csv, city_id):
    test_df = pd.read_csv(csv, na_values=["?", " ", "NA", "na"])
    test = test_df[test_df["city"] == int(city_id)].copy()
    return test    

In [93]:
def get_path_prefix(city_id, data_type="train"):
    path_prefix = "./split_files/city_" + str(city_id) + "_files/simple_" + data_type + "_"
    return path_prefix

In [94]:
def add_date_column(df): #faster way to stitch year month and day together into date
    df["date"] =  list(zip(df["year"], df["month"], df["day"]))
    df["date"] = df["date"].astype("category")
    (year, month, day) = zip(*(df["date"].cat.categories))
    new_cats = [str(year[i])+"-"+str(month[i])+"-"+str(day[i]) for i in range(0, len(day))]
    new_cats_map = dict(zip(df["date"].cat.categories, new_cats))
    df["date"]=df["date"].map(new_cats_map)
    df["date"] = pd.to_datetime(df["date"])
    df["date"] = df["date"].astype("datetime64")    
    df.drop(["year", "month", "day"], axis=1, inplace=True)
    return df

def add_city_med_column(df): #faster way to create "city_med" column
    df["city_med"] =  list(zip(df["city"], df["medicine"]))
    df["city_med"] = df["city_med"].astype("category")
    (city, med) = zip(*(df["city_med"].cat.categories))
    new_cats = [str(city[i])+"_"+str(med[i]) for i in range(0, len(city))]
    new_cats_map = dict(zip(df["city_med"].cat.categories, new_cats))
    df["city_med"]=df["city_med"].map(new_cats_map)
    return df

def add_discounted_column(df, discounted_df):
    out_df = df.copy()
    out_df = pd.merge(out_df, discounted, how = "left", left_on = ["date", "city", "medicine"],
         right_on = ["date", "city", "medicine"])
    out_df["discounted"] = out_df["discounted"].fillna(0)
    out_df["discounted"] = out_df["discounted"].astype("int64")
    return (out_df)

def add_footfall_column(df, footfall_df):
    out_df = pd.merge(df, footfall_df, how = "left", left_on = ["date", "city"], right_on = ["date", "city"])
    out_df["footfall"] = out_df["footfall"].fillna(0)
    return out_df

def fill_missing_dates(df, date_col, start_date = "", end_date = ""):
    df[date_col] = pd.to_datetime(df[date_col])
    dates = pd.DataFrame(pd.date_range(start = start_date, end = end_date, freq='D') )
    dates = dates.rename(columns = {0:"date"})
    df = pd.merge(df, dates, how="outer", left_on=["date"], right_on=["date"])
    df = df.fillna(0)
    return df

In [95]:
def split_data(df, splitby, prefix, start_date, end_date, footfall):
    df = add_date_column(df)
    df = add_city_med_column(df)
    footfall_copy = footfall.copy()
    footfall_copy = add_date_column(footfall_copy)
    df[splitby]=df[splitby].astype("category")
    for l in list(df[splitby].cat.categories):
        fname = prefix + "_" + str(l) + ".csv"
        l_df = df[df[splitby] == l].reset_index()
        city = l_df.loc[0, "city"]
        med = l_df.loc[0, "medicine"]
        l_df = l_df.drop(["city", "medicine", "city_med"], axis = 1)
        l_df = fill_missing_dates(l_df, "date", start_date, end_date)
        l_df["city"] = city
        l_df["medicine"] = med
        l_df = add_discounted_column(l_df, discounted)
        l_df = add_footfall_column(l_df, footfall_copy)
        l_df["index"] = l_df["index"].astype("int64")
        l_df["real"] = (l_df["index"] > 0).astype(int)
        l_df = l_df.drop(["index"], axis = 1)
        l_df.to_csv(fname,index=False)

In [96]:
city_dict = read_city_dict("city_dict.json")
test = read_test_csv("test_data.csv", 6)
train = read_train_csv("train_data.csv", 6)
discounted = read_discount_features_csv("discount_features.csv")
footfall_compact = pd.read_csv("footfall_compact.csv", na_values=["?", " ", "NA", "na"])

In [99]:
train_prefix = get_path_prefix(6, "train")
test_prefix = get_path_prefix(6, "test")

In [100]:
split_data(train, "city_med", train_prefix, start_date = "2015-01-02", end_date = "2018-06-30", footfall = footfall_compact)
split_data(test, "city_med", test_prefix, start_date = "2018-07-01", end_date = "2018-07-31", footfall = footfall_compact)